In [17]:

import numpy as np
from ortools.sat.python import cp_model as cp
import gurobipy as gp
from collections import deque
from itertools import chain

In [18]:
def readGrid(path):
    with open(f"../assets/data/Fobidoshi/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("12x12_1")
    for g in grid:
        print(g)

['X', 'O', '.', 'O', 'X', '.', '.', '.', 'X', '.', 'O', 'O']
['O', '.', 'O', '.', 'O', 'O', '.', '.', '.', '.', '.', '.']
['.', 'O', 'O', '.', 'O', '.', '.', 'O', '.', 'O', '.', '.']
['O', '.', '.', '.', '.', '.', '.', '.', 'O', '.', 'O', 'X']
['.', '.', 'O', 'X', 'O', '.', '.', '.', 'X', '.', '.', '.']
['X', '.', 'O', '.', '.', '.', '.', '.', '.', '.', 'O', '.']
['O', '.', '.', '.', '.', '.', 'O', 'O', '.', '.', '.', 'X']
['O', '.', 'O', 'X', '.', '.', '.', '.', 'X', '.', '.', '.']
['X', '.', '.', '.', '.', '.', '.', 'O', '.', '.', '.', 'O']
['.', '.', 'O', '.', '.', 'O', 'O', 'O', '.', '.', '.', 'O']
['O', '.', 'O', 'O', '.', 'O', 'O', 'O', '.', '.', '.', 'O']
['O', '.', '.', 'X', 'O', '.', 'O', 'X', 'O', '.', '.', 'X']


In [35]:
def FobidoshiSolver(m, n, grid) :
    """_summary_

    Args:
        m (_type_): 行数
        n (_type_): 列数
        grid (_type_): 网格
    """
    
    Fobidoshi = gp.Model("Fobidoshi")
    Fobidoshi.modelSense = gp.GRB.MAXIMIZE
    Fobidoshi.Params.lazyConstraints = 1
    Fobidoshi.update()
    x = {}
    for i in range(m):
        for j in range(n):
            x[i, j] = Fobidoshi.addVar(
            vtype = gp.GRB.BINARY,
            obj = 1,
            name = f"x[{i},{j}]")
            if grid[i][j] == "X":
                Fobidoshi.addConstr(x[i, j] == 1)
            elif grid[i][j] == "O":
                Fobidoshi.addConstr(x[i, j] == 0)
    for i in range(m):
        for j in range(n - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i, j + 1], x[i, j + 2], x[i, j + 3]]) >= 1, name = f"x[{i},{j},r]")
    for j in range(n):
        for i in range(m - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i + 1, j], x[i + 2, j], x[i + 3, j]]) >= 1, name = f"x[{i},{j},c]")
    
    for i in range(m):
        for j in range(n):
            neighbours = []
            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            for (subx, suby) in directions:
                if (i + subx >= 0 and i + subx < m) and (j + suby >= 0 and j + suby < n):
                    neighbours.append((i + subx, j + suby))
            Fobidoshi.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in neighbours) <= len(neighbours) - 1 + x[i, j], name = f"alone_{i}_{j}")
    
    Fobidoshi.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MAXIMIZE)
    # Fobidoshi.write("Fobidoshi.lp")
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for _ in range(m)]
            for i in range(m):
                for j in range(n):
                    if x_sol[i, j] > 1e-6:
                        curr_grid[i][j] = 1
                    else:
                        curr_grid[i][j] = 0
            
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white =  rows * cols - sum(chain.from_iterable(grid))
                # for g in grid:
                #     print(g)
                # print("====")
                print(f"total vacant is {total_white}")
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 0 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = [] # record cells that formulate the border: list(tuple)
                cells_inside_borders = [] # record all cells inside the border: list(tuple)
                cnt_white = 0
                while len(visited) < total_white :
                    start = find_unvisited_white()
                    if not start:
                        break
                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    current_inside_cells = [] # record cells in current border
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        current_inside_cells.append((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 0 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 1:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))
                    cells_inside_borders.append(current_inside_cells)
                    
                assert len(borders_cut) == len(cells_inside_borders)

                if len(visited) == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    
                    return [], []
                return borders_cut, cells_inside_borders
            
            borders_cut, cells_inside_borders = find_surrounding_black_cells(curr_grid)

            for borders, cells_inside in zip(borders_cut, cells_inside_borders):
                
                if len(cells_inside) < m * n // 2:
                # if len(borders) < m * n // 4:
                    for (cell_x, cell_y) in cells_inside:
                        # print("CUT ", end = " ")
                        # print(borders, (cell_x, cell_y))
                        model.cbLazy(gp.quicksum(model._x[subx, suby] for (subx, suby) in borders ) <= len(borders) - 1 + model._x[cell_x, cell_y])

    Fobidoshi._x = x
    Fobidoshi.optimize(border_elim)
    # Fobidoshi.computeIIS()
    # Fobidoshi.write("fobidoshi.ilp")
    # Visualize 
    for i in range(m):
        for j in range(n):
            if x[i, j].x > 1e-6:
                print(f"X", end = " ")
            else:
                print(f"O", end = " ")
        print()
    print()

    
if __name__ == "__main__":
    m, n, grid = readGrid("12x12_4")
    FobidoshiSolver(m, n, grid)
    

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 406 rows, 144 columns and 1582 nonzeros
Model fingerprint: 0x4ab98b48
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 282 rows and 63 columns
Presolve time: 0.00s
Presolved: 124 rows, 81 columns, 381 nonzeros
Variable types: 0 continuous, 81 integer (81 binary)
total vacant is 66

Root relaxation: objective 7.900000e+01, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   79.00000    0    6          -   79.00000      -     

total vacant is 78
     0     2   70.75000    0   31          -   70.75000      -     -    0s
total vacant is 77
total vacant is 84
total vacant is 79
total vacant is 79
total vacant is 81
total vacant is 83
total vacant is 83
total vacant is 83
total vacant is 84
total vacant is 85
total vacant is 83
total vacant is 86
total vacant is 87
total vacant is 84
total vacant is 88
total vacant is 85
total vacant is 85
total vacant is 88
total vacant is 90
total vacant is 88
total vacant is 90
total vacant is 91
total vacant is 88
total vacant is 88
total vacant is 86
total vacant is 85
total vacant is 86
total vacant is 92
total vacant is 89
total vacant is 87
total vacant is 88
total vacant is 89
total vacant is 93
total vacant is 87
total vacant is 88
total vacant is 91
total vacant is 89
total vacant is 88
total vacant is 91
total vacant is 91
total vacant is 91
total vacant is 92
total vacant is 93
total vacant is 92
total vacant is 95
total vacant is 91
total vacant is 92
total vacant 

In [26]:
def parse_data(input_str):
    # Strip spaces and find the index of the last opening parenthesis
    last_parenthesis = input_str.rfind('(')
    
    # Extract the list part and the last tuple
    list_part = input_str[1:last_parenthesis-2]  # remove the enclosing brackets
    tuple_part = input_str[last_parenthesis:]
    
    # Convert the string representations to actual list and tuple
    list_of_tuples = eval(f"[{list_part}]")
    coordinate = eval(tuple_part)
    
    return list_of_tuples, coordinate

RES =[ \
["X", "O",  "O",  "O",  "X",  "O",  "O",  "O",  "X",  "X",  "O", "O" ],
["O", "X",  "O",  "X",  "O",  "O",  "X",  "O",  "O",  "O",  "X", "O" ],
["O", "O",  "O",  "X",  "O",  "X",  "O",  "O",  "X",  "O",  "O", "O" ],
["O", "O",  "X",  "X",  "X",  "O",  "O",  "X",  "O",  "O",  "O", "X"],
["X", "O",  "O",  "X",  "O",  "O",  "X",  "X",  "X",  "X",  "X", "O" ],
["X", "X",  "O",  "O",  "X",  "O",  "O",  "X",  "O",  "O",  "O", "X" ],
["O", "X",  "X",  "O",  "O",  "X",  "O",  "O",  "O",  "X",  "O", "X" ],
["O", "O",  "O",  "X",  "O",  "O",  "O",  "X",  "X",  "O",  "O", "X" ],
["X", "O",  "X",  "O",  "O",  "X",  "X",  "O",  "O",  "O",  "X", "O" ],
["X", "O",  "O",  "O",  "X",  "O",  "O",  "O",  "X",  "O",  "O", "O" ],
["O", "X",  "O",  "O",  "X",  "O",  "O",  "O",  "X",  "X",  "O", "O" ],
["O", "O",  "O",  "X",  "O",  "O",  "O",  "X",  "O",  "O",  "O", "X"]]

a = [[0] * 12 for _ in range(12)]
xx = {}
for i in range(12):
    for j in range(12):
        if RES[i][j] == "O":
            xx[i, j] = 0
        else:
            xx[i, j] = 1

for line in res:
    list_of_tuples, cd = parse_data(line.strip())
    if not sum(xx[i, j] for (i, j) in list_of_tuples) <= len(list_of_tuples) - 1 + xx[cd[0], cd[1]]:
        print("\n === \n")
        print(list_of_tuples)
        print("-----")
        print(cd)
        # print(sum(xx[i, j] for (i, j) in list_of_tuples) <= len(list_of_tuples) - 1 + xx[cd[0], cd[1]])


 === 

[(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (4, 6), (5, 1), (5, 7), (8, 0), (9, 8), (8, 6), (10, 9), (1, 6), (0, 8), (2, 5), (1, 3), (11, 11), (2, 8), (6, 2), (7, 7), (6, 5), (6, 11), (5, 0), (4, 8), (8, 2), (8, 5), (9, 4), (11, 7), (0, 4), (10, 8), (6, 1), (7, 3), (3, 2), (3, 11), (4, 10), (9, 0), (5, 11), (0, 0), (10, 4), (1, 1), (11, 3), (10, 1), (8, 10), (0, 9), (2, 3), (1, 10), (7, 11), (6, 9), (7, 8)]
-----
(0, 1)

 === 

[(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (4, 6), (5, 1), (5, 7), (8, 0), (9, 8), (8, 6), (10, 9), (1, 6), (0, 8), (2, 5), (1, 3), (11, 11), (2, 8), (6, 2), (7, 7), (6, 5), (6, 11), (5, 0), (4, 8), (8, 2), (8, 5), (9, 4), (11, 7), (0, 4), (10, 8), (6, 1), (7, 3), (3, 2), (3, 11), (4, 10), (9, 0), (5, 11), (0, 0), (10, 4), (1, 1), (11, 3), (10, 1), (8, 10), (0, 9), (2, 3), (1, 10), (7, 11), (6, 9), (7, 8)]
-----
(0, 2)

 === 

[(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (4, 6), (5, 1), (5, 7), (8, 0), (9, 8), (8, 6), (10, 9), (1, 6), (0, 

In [24]:
m, n, grid = readGrid("12x12_1")
for g in grid:
    print(g)

for i in range(m):
    for j in range(n - 3):
        print(f"r_{i}_{j}_", end = " ")
        print(sum([xx[i, j], xx[i, j + 1], xx[i, j + 2], xx[i, j + 3]]) >= 1)
for j in range(n):
    for i in range(m - 3):
        print(f"c_{i}_{j}_", end = " ")
        print(sum([xx[i, j], xx[i + 1, j], xx[i + 2, j], xx[i + 3, j]]) >= 1)

# for i in range(m):
#     for j in range(n):
#         neighbours = []
#         directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
#         for (subx, suby) in directions:
#             if (i + subx >= 0 and i + subx < m) and (j + suby >= 0 and j + suby < n):
#                 neighbours.append((i + subx, j + suby))
#         Fobidoshi.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in neighbours) <= len(neighbours) - 1 + x[i, j], name = f"alone_{i}_{j}")

['X', 'O', '.', 'O', 'X', '.', '.', '.', 'X', '.', 'O', 'O']
['O', '.', 'O', '.', 'O', 'O', '.', '.', '.', '.', '.', '.']
['.', 'O', 'O', '.', 'O', '.', '.', 'O', '.', 'O', '.', '.']
['O', '.', '.', '.', '.', '.', '.', '.', 'O', '.', 'O', 'X']
['.', '.', 'O', 'X', 'O', '.', '.', '.', 'X', '.', '.', '.']
['X', '.', 'O', '.', '.', '.', '.', '.', '.', '.', 'O', '.']
['O', '.', '.', '.', '.', '.', 'O', 'O', '.', '.', '.', 'X']
['O', '.', 'O', 'X', '.', '.', '.', '.', 'X', '.', '.', '.']
['X', '.', '.', '.', '.', '.', '.', 'O', '.', '.', '.', 'O']
['.', '.', 'O', '.', '.', 'O', 'O', 'O', '.', '.', '.', 'O']
['O', '.', 'O', 'O', '.', 'O', 'O', 'O', '.', '.', '.', 'O']
['O', '.', '.', 'X', 'O', '.', 'O', 'X', 'O', '.', '.', 'X']
r_0_0_ True
r_0_1_ True
r_0_2_ True
r_0_3_ True
r_0_4_ True
r_0_5_ True
r_0_6_ True
r_0_7_ True
r_0_8_ True
r_1_0_ True
r_1_1_ True
r_1_2_ True
r_1_3_ True
r_1_4_ True
r_1_5_ True
r_1_6_ True
r_1_7_ True
r_1_8_ True
r_2_0_ True
r_2_1_ True
r_2_2_ True
r_2_3_ True
r_2_